<a href="https://colab.research.google.com/github/ItzWarLockOp/Google-Colab-Scriptz/blob/main/Sync%20Multiple%20Drives%20at%20Once.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <center><img src='https://raw.githubusercontent.com/ItzWarLockOp/Google-Colab-Scriptz/main/png/IMG_20230604_021032.png' height="83" alt="SyncMulti-Drive"/>

#<h4> <b>[Step : 1️⃣]

#<h3><b><font color='orange'>📲Install Rclone & Dependencies 📚

In [ ]:
#@title <b>⬅️Run this cell to Install Rclone📲

from sys import exit as end
from pathlib import Path
from requests import get as get_url
from google.colab import output
from json import dumps as dump_json

current_version = "v5.1.0"
do_normal_setup = "1"

# Check for latest version
try:
    print("Connecting to CDN...")
    json_data = get_url("https://raw.githubusercontent.com/ItzWarLockOp/Trash/main/mct.json").json()
    output.clear()
    print("Connected with CDN.")
    if current_version == json_data["version"]:
        print("Checking current version...")
        print(f'Great Job! You are using the latest version: {json_data["version"]}')
    else:
        print("Checking current version...")
        print(f'Oh-No! You are not using the latest version and missing out on important fixes & features. The latest version is {json_data["version"]}. Please use the latest notebook available here:\n{json_data["notebookLink"]}')
except Exception as error_code:
    end(f'Failed to connect with CDN!\nTraceback:\n{error_code}')

# Save json data locally
with open('mct.json', 'w+') as mct_json:
    mct_json.write(dump_json(json_data))


# User Input
INSTALL_RCLONE_FLAVOR = ""

# Install rclone
try:
    print("Installing rclone...")
    !curl -s https://rclone.org/install.sh | sudo bash {INSTALL_RCLONE_FLAVOR} > /dev/null 2>&1
    output.clear()
    print("Rclone installation completed!")
except Exception as error_code:
    print(f'Failed to install rclone!\nTraceback:\n{error_code}')

if do_normal_setup == '1' and not Path("/content/setup.sh").exists():
    try:
        print("Setting up Libraries...")
        get_url(json_data["setupFile"])
        !curl -s {json_data["setupFile"]} | sudo bash {INSTALL_RCLONE_FLAVOR} > /dev/null 2>&1
        !curl {json_data["setupFile"]} > "/content/setup.sh"
        output.clear()
        print("Setup completed!")
    except ConnectionError:
        print('Unable to get setup file!')
    except Exception as error_code:
        print(f'Setup instructions seem to have mistakes!\nTraceback:\n{error_code}')
elif do_normal_setup == '1' and Path("/content/setup.sh").exists():
    output.clear()
    print("Setup file exists and is already configured!")

In [ ]:

#@title <b>⬅️Run this cell to Install Dependencies📲


print("Installing Dependencies...", end='')
print(".", end='', flush=True)
!pip install yt-dlp > /dev/null 2>&1

!pip install -q tqdm
!pip install -q google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2

from google.colab import auth
from googleapiclient.discovery import build

# Authenticate and create the Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3')

print("User Authenticated Successfully!")

import IPython.display as display
!pip install --quiet --disable-pip-version-check libtorrent
display.display(display.Markdown("### libtorrent installation sucessful!"))
display.clear_output(wait=True)

print("Installation Finished ! ✅")

#<h4> <b>[Step : 2️⃣]

#<center><img src="https://cdn.thecaduceus.eu.org/MCT/img/rclone_logo.png" height="50" alt="Rclone-Logo"/></center><center><h3> <font color='lime'><b> Upload [Rclone.Conf] File</b></h3></center>

In [ ]:
#@markdown <b>  ⬅️ Run this Cell to Initiate Upload 📲
from google.colab import files
from sys import exit as end
from pathlib import Path, PurePath
from IPython.display import clear_output

UPLOAD_CONFIG_FILE = True

if UPLOAD_CONFIG_FILE:
    # Initiate Upload Request
    print('Request: Initiate upload -> rclone.conf [file]')
    uploaded_files = files.upload()
    if len(uploaded_files) == 0:
        end('File Error: No file uploaded')

    uploaded_file_name = next(iter(uploaded_files))
    if uploaded_file_name != 'rclone.conf':
        end('File Error: File is not rclone.conf')

    file_content = uploaded_files[uploaded_file_name]
    rclone_conf_path = Path("/root/.config/rclone/")
    rclone_conf_path.mkdir(parents=True, exist_ok=True)
    rclone_conf_file_path = rclone_conf_path / "rclone.conf"
    with open(rclone_conf_file_path, 'wb') as f:
        f.write(file_content)

    print('Rclone.conf uploaded successfully')

# Clear output and show last print message
clear_output(wait=True)
print('Rclone config File uploaded!')

#<h4> <b>[Step : 3️⃣]

#<center> <h3> <font color='sky blue'> <b>Mount & Unmount All Remotes 📁

In [ ]:

from sys import exit as end
from os import makedirs, system
from pathlib import Path, PurePath
from re import findall, M

if not Path("/root/.config/rclone/rclone.conf").exists():
    end('Not Found: rclone.conf [file]')

content = open("/root/.config/rclone/rclone.conf").read()
avCon = findall(r"^\[(.+)\]$", content, M)

print("Available Rclone Remotes:", end=" ")
print(*avCon, sep=", ")

if len(avCon) == 0:
    end('No remotes found in rclone.conf [file]')

Mount_Point_Base = "/content/drives"
Cache_dir = "/content/temp/Rclone"

Action = "mount"  #@param ["mount", "unmount"] {type:"string"}

if Action not in ["mount", "unmount"]:
    end("Input Error: Value should be 'mount' or 'unmount'.")

if Action == "mount":
    for remote in avCon:
        Mount_Point = f"{Mount_Point_Base}/{remote}"

        makedirs(Mount_Point, exist_ok=True)
        mount_command = (
            f"rclone mount {remote}: {Mount_Point} --user-agent 'Mozilla' "
            "--buffer-size 256M --transfers 10 --vfs-cache-mode minimal "
            "--vfs-read-chunk-size 500M --vfs-cache-max-size 50G "
            "--vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s "
            f"--cache-dir {Cache_dir} --allow-other --daemon"
        )
        system(mount_command)

        if Path(Mount_Point).exists():
            print(f'{remote} mounted: {Mount_Point}')
        else:
            print(f"Mount Error: Unable to mount {remote} at {Mount_Point}")
elif Action == "unmount":
    for remote in avCon:
        Mount_Point = f"{Mount_Point_Base}/{remote}"
        unmount_command = f"umount -l {Mount_Point}"  # Use "umount" instead of "fusermount"
        if system(unmount_command) == 0:
            print(f'{remote} unmounted')
        else:
            print(f"Unable to unmount {remote}")

#<h1><center> <b> <font color= 'pink'>★Multi-functions★

<h1><center> <b>↑↓Direct Link Downloader↑↓

In [ ]:
#@title <center> <b><h5> • Please Make Sure To Fill All The Info • </b></h5></center>
#@markdown ------

import os
import requests
from tqdm import tqdm
import math

# Get user input for the download link and path
DownloadLink = "https://DirectDownloadLink.com/Video.mkv" #@param {type:"string"}
Path = "/content/drive/MyDrive/" #@param {type:"string"}

# Set the desired new name for the file
NAMEwithEXTENSION = "Video.mkv" #@param {type:"string"}

#@markdown ------


# Print the file name and save path
print("File Name:", NAMEwithEXTENSION)
print("Save Path:", os.path.join(Path, NAMEwithEXTENSION))
print()

# Check if NAMEwithEXTENSION is empty
if not NAMEwithEXTENSION:
    print("Please enter a name with an appropriate extension.")
    Download = False
else:
    # Check if extension is present
    if '.' not in NAMEwithEXTENSION:
        print("Please enter an appropriate extension to download the file.")
        Download = False
    else:
        Download = True

if Download:
    # Check if the file path already exists
    file_path = os.path.join(Path, NAMEwithEXTENSION)
    file_exists = os.path.exists(file_path)
    if file_exists:
        print("A file with the same name already exists in the specified path. Download process aborted.")
        Download = False

if Download:
    # Download the file and save it to the specified path with the new filename
    response = requests.get(DownloadLink, stream=True)
    total_size = int(response.headers.get("content-length", 0))

    # Convert the total size to a human-readable format
    size_units = ["B", "KB", "MB", "GB", "TB"]
    size_index = int(math.floor(math.log(total_size, 1024)))
    total_size_hr = f"{round(total_size / (1024 ** size_index), 2)} {size_units[size_index]}"

    # Print the file size
    print("File Size:", total_size_hr)
    print()

    # Create the save directory if it doesn't exist
    os.makedirs(Path, exist_ok=True)

    # Download the file with progress bar
    with open(file_path, "wb") as file:
        progress_bar = tqdm(total=total_size, unit="B", unit_scale=True, ncols=80, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')
        for data in response.iter_content(chunk_size=1024):
            file.write(data)
            progress_bar.update(len(data))
        progress_bar.close()

    # Add an extra line space
    print("\n")

    # Print completion message
    print("📲 File Downloaded Successfully! 🎉")

___________________________________________

<h1><center> <b> Copy Folders 📂 Between Drive2Drive

In [ ]:

from tqdm import tqdm

def copy_shared_folder_to_target_folder(shared_folder_path, destination_path):
    # Get the shared folder ID from the path
    query = f"name='{shared_folder_path.split('/')[-1]}' and mimeType='application/vnd.google-apps.folder'"
    shared_folder = drive_service.files().list(q=query, fields='files(id)').execute()
    shared_folder_id = shared_folder['files'][0]['id']
    shared_folder_name = shared_folder_path.split('/')[-1]

    # Get the destination folder ID from the path
    query = f"name='{destination_path.split('/')[-1]}' and mimeType='application/vnd.google-apps.folder'"
    destination_folder = drive_service.files().list(q=query, fields='files(id)').execute()
    destination_folder_id = destination_folder['files'][0]['id']

    # Create a folder with the same name in the destination folder
    folder_metadata = {
        'name': shared_folder_name,
        'parents': [destination_folder_id],
        'mimeType': 'application/vnd.google-apps.folder'
    }
    new_folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    new_folder_id = new_folder.get('id')

    # Copy the contents of the shared folder to the new folder
    query = f"'{shared_folder_id}' in parents"
    shared_folder_contents = drive_service.files().list(q=query, fields='files(id)').execute()
    num_files = len(shared_folder_contents.get('files', []))

    with tqdm(total=num_files, desc='Copying files') as pbar:
        for file in shared_folder_contents.get('files', []):
            file_id = file.get('id')
            drive_service.files().copy(fileId=file_id, body={'parents': [new_folder_id]}).execute()
            pbar.update(1)

    print('\nShared folder copied successfully!🎉')

shared_folder_path = '/content/drive/MyDrive/Shared Folder' #@param {type:"string"}
destination_path = '/content/drive/MyDrive/Destination' #@param {type:"string"}

copy_shared_folder_to_target_folder(shared_folder_path, destination_path)

___________________________________________

<h1><center> <b> 🧲Libtorrent

In [ ]:

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import libtorrent as lt
import time
from tqdm.auto import tqdm
import requests
import tempfile
import os

ses = lt.session()
ses.listen_on(6881, 6891)
ses.start_dht()  # Enable DHT
ses.set_max_connections(100)  # Set connection limit to 100

downloads = []

download_path = "/content/X"  #@param {type:"string"}
params = {
    "save_path": download_path,
    "storage_mode": lt.storage_mode_t(2)
}

link = "magnet:?xt=urn:enteryourmagnethere" #@param {type:"string"}

if "magnet:" in link:
    handle = lt.add_magnet_uri(ses, link, params)
else:
    # Download the .torrent file from the URL
    response = requests.get(link)
    temp_file_path = "/content/temp.torrent"  # Set the temporary file path in Colab
    with open(temp_file_path, 'wb') as temp_file:
        temp_file.write(response.content)
    with open(temp_file_path, 'rb') as file:
        data = file.read()
        info = lt.torrent_info(lt.bdecode(data))
    handle = lt.add_magnet_uri(ses, lt.make_magnet_uri(info), params)
    os.remove(temp_file_path)

print("Downloading metadata...")

while not handle.has_metadata():
    time.sleep(1)

print("Starting", handle.name())

# Get initial peer list
peers = handle.get_peer_info()

with tqdm(total=handle.status().total_wanted, unit='B', unit_scale=True, desc=handle.name()) as pbar:
    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata',
                     'downloading', 'finished', 'seeding', 'allocating']
        pbar.set_postfix({'State': state_str[s.state], 'Seeding': s.num_complete, 'Leeching': s.num_incomplete})
        downloaded = s.total_wanted_done
        pbar.update(downloaded - pbar.n)
        pbar.set_description(f"{handle.name()} - {s.download_rate/1000:.1f} KB/s - {s.progress*100:.2f}%")

        # Check and switch to a seeder with higher download speed
        max_speed = 0
        max_speed_peer = None
        for peer in peers:
            download_rate = (peer.total_download - peer.total_upload) / (time.time() - peer.last_active)
            if download_rate > max_speed:
                max_speed = download_rate
                max_speed_peer = peer

        if max_speed_peer:
            endpoint = max_speed_peer.ip
            handle.connect_peer(endpoint)

        time.sleep(1)

        # Update peer list
        peers = handle.get_peer_info()

print("Download Completed!")

___________________________________________

<h1><center> <b> ★M3U8★ ↑↓DOWNLOADER↑↓

In [ ]:

import os
import subprocess
import time
import re
import shutil
from IPython.display import clear_output

def download_video(VideoURL, SavePath, options=None, NewName=None):
    command = ["yt-dlp", VideoURL, "-o", SavePath]
    if options:
        command.extend(options)

    log_file = "/content/download.log"
    with open(log_file, "w") as f:
        process = subprocess.Popen(command, stdout=f)
        while process.poll() is None:
            time.sleep(1)
            with open(log_file, "r") as f_read:
                output = f_read.read().strip()
                progress_match = re.findall(r"\d+\.\d+%", output)
                if progress_match:
                    progress = float(progress_match[-1].strip("%"))
                    clear_output(wait=True)
                    print(f"Progress: {progress}%")
        # Read any remaining output after the process finishes
        with open(log_file, "r") as f_read:
            output = f_read.read().strip()
            progress_match = re.findall(r"\d+\.\d+%", output)
            if progress_match:
                progress = float(progress_match[-1].strip("%"))
                clear_output(wait=True)
                print(f"Progress: {progress}%")

    # Remove the log file
    os.remove(log_file)

    # Rename the output file if a new name is provided
    if NewName:
        if not NewName.endswith(".mp4"):
            NewName += ".mp4"
        renamed_output_path = os.path.join(os.path.dirname(SavePath), NewName)
        os.rename(SavePath, renamed_output_path)
        SavePath = renamed_output_path

    # Move the output file to the provided Google Drive path
    DownloadPath = "/content/drive/MyDrive/"  #@param {type:"string"}
    destination_path = os.path.join(DownloadPath, os.path.basename(SavePath))
    os.makedirs(os.path.dirname(destination_path), exist_ok=True)
    shutil.move(SavePath, destination_path)
    print("File Downloaded Successfully!🎉")

# Example usage:
VideoURL = "https://yourlink.com/hls=stream.m3u8" #@param {type:"string"}
output_file = "/content/output.mp4"
download_options = ["-f", "best"]
NewName = "Video" #@param {type:"string"}

download_video(VideoURL, output_file, download_options, NewName=NewName)

___________________________________________

<img src='https://raw.githubusercontent.com/yt-dlp/yt-dlp/master/.github/banner.svg' height="83" alt="Youtube-DL"/>

In [ ]:
#@title <b> Enter Path, Url and Quality 📡


import yt_dlp
import os

# Set the download directory to your Google Drive
save_path = '/content/drive/MyDrive/Youtube Videos' #@param {type:"string"}

# Create the download directory if it doesn't exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Define a function to download a video
def download_video(youtube_url, quality):
    ydl_opts = {
        'outtmpl': save_path + '/%(title)s.%(ext)s',
        'format': f'bestvideo[height<={quality}][ext=mp4]+bestaudio[ext=m4a]/mp4',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])
    print("Download completed!")

# Example usage: download a video and save it to your Google Drive
youtube_url = 'https://www.youtube.com/watch?v=dQw4w9WgXcQ' #@param {type:"string"}
quality = "2160" #@param ["2160","1440","1080","720","480","360","240","144"]
download_video(youtube_url, quality)

___________________________________________

<center><img src='https://i.ibb.co/XVzcc3B/IMG-20230603-183942.png' height="90" alt="7zip"/>

<h2> <b><font color ='Orange'>🔒[Archive Files or Folder]🔒

<h3><b><center>• Enter Source Path •</h3></b>
<h3><b><center>• Enter Archive Format •</h3></b>
<h3><b><center>• Enter Password to Encrypt the file •</h3></b>
<h3><b><center>• Select Compression Level •</h3></b>
<h3><b><center>• Enter Output Path • </h3></b>

Note :
* If <b>IncludeDirectory</b> is True then All Files + Including the folder they are in will be Archived.

* If <b>IncludeDirectory</b> is False the only just files inside the folder going to be archived.

In [ ]:
import os

sourcePath = "/content/drive/MyDrive/Files" #@param {type:"string"}
compressType = "zip" #@param ["zip", "7z", "tar", "gzip", "bzip2", "xz", "wim"]
includeDirectory = True #@param {type:"boolean"}
Password = "" #@param {type:"string"}
Split = "no" #@param ["no","5m", "10m", "100m", "500m", "1g", "2g"] {allow-input: true}
compressionLevel = "No Compression" #@param ["No Compression", "Fastest", "Fast", "Normal", "Maximum", "Ultra"]
saveToSourceLocation = False #@param {type:"boolean"}

compressionLevels = {
    "No Compression": "0",
    "Fastest": "1",
    "Fast": "3",
    "Normal": "5",
    "Maximum": "7",
    "Ultra": "9"
}

commandLine = "-t" + compressType + " -mx=" + compressionLevels[compressionLevel]

if len(Password) > 0:
    commandLine = commandLine + " -p" + '"' + Password + '"'

if Split != "no":
    commandLine = commandLine + " -v" + '"' + Split + '"'

if os.path.isfile(sourcePath):
    sourceName = os.path.splitext(os.path.basename(os.path.abspath(sourcePath)))[0]
    sourceFolder = os.path.dirname(os.path.abspath(sourcePath))
else:
    sourceName = os.path.split(os.path.abspath(sourcePath))[1]
    sourceFolder = os.path.split(os.path.abspath(sourcePath))[0]

if saveToSourceLocation:
    outputPath = sourceFolder
else:
    outputPath = "/content/drive/MyDrive/" #@param {type:"string"}

if includeDirectory:
    compressPath = os.path.join(outputPath, sourceName)
else:
    compressPath = outputPath

commandLine = commandLine + ' "' + compressPath + '"'

if includeDirectory:
    commandLine = commandLine + ' "' + sourcePath + '"'
else:
    commandLine = commandLine + ' "' + sourcePath + '"/*'

if compressType == "gzip":
    formatFile = "gz"
elif compressType == "bzip2":
    formatFile = "bz2"
else:
    formatFile = compressType

!7z a {commandLine}
saveFile = compressPath + '.' + formatFile

if os.path.exists(saveFile):
    if Split != "no":
        baseName = os.path.splitext(saveFile)[0]
        splitFiles = baseName + '.z*'
        os.rename(splitFiles, baseName + '.zip')
        saveFile = baseName + '.zip'

<h2> <b><font color ='Orange'>🔓[UnArchive Files]🔓

<h3><b><center>• Enter File Path •</h3></b>
<h3><b><center>• Enter Password if file is Encrypted •</h3></b>
<h3><b><center>• Enter Output Path •</h3></b>

Note :

* If the <b>viewfile</b> is True then Unarchive will not be proceed.

* If you want to unarchive the file to same location the source exist then set <b>saveToSourceLocation</b> to True

In [ ]:
import os

sourceFile = "/content/File.zip"  # @param {type:"string"}
viewFile = False  # @param {type:"boolean"}
password = ""  # @param {type:"string"}
saveToSourceLocation = False  # @param {type:"boolean"}


sourceFolder = os.path.dirname(os.path.abspath(sourceFile))

if viewFile is True:
    commandLine = 'l "' + sourceFile + '"'
else:
    if saveToSourceLocation is True:
        extractFolder = sourceFolder
    else:
        outputPath = "/content/"  # @param {type:"string"}
        extractFolder = outputPath

    commandLine = 'x "' + sourceFile + '" -o"' + extractFolder + '"'
    if password:
        commandLine += ' -p' + password

    # Add auto-skip option
    commandLine += ' -aoa'

output = !7z {commandLine}

___________________________________________

<center><img src='https://i.ibb.co/qgbykgw/IMG-20230531-162006.png%22' height="90" alt="Renamer"/>

<h3><b><font color="yellow">📲Enter Folder Path & New Name📁

In [ ]:

import os
from tqdm import tqdm

def rename_files(folder_path, new_name):
    file_names = os.listdir(folder_path)
    num_files = min(9999, len(file_names))
    pbar = tqdm(total=num_files, desc='Renaming Files', unit='file', bar_format='{desc}: {percentage:.0f}% {n_fmt}/{total_fmt}')

    for count, file_name in enumerate(file_names, start=1):
        if count > num_files:
            break

        file_extension = os.path.splitext(file_name)[1]
        new_file_name = f'{new_name} [{str(count).zfill(4)}]{file_extension}'

        old_file_path = os.path.join(folder_path, file_name)
        new_file_path = os.path.join(folder_path, new_file_name)

        os.rename(old_file_path, new_file_path)
        pbar.update(1)

    pbar.close()

# Example usage:
folder_path = '/content/drive/MyDrive/X'  #@param {type:"string"}
new_name = 'Images'  #@param {type:"string"}
rename_files(folder_path, new_name)